In [8]:
import pandas as pd
import numpy as np
import requests
import json
import os

In [9]:
data_folder_path = 'C:\\Users\\chait\\OneDrive\\Documents\\School\\IFT_6758\\2.Project\\Milestone2_Draft\\data_phase2\\'

In [10]:
def get_file(game_id, folder_path):
    """
    Input:
    game_id: string, formed using the season,type and the game number
    folder_path: destination path to save the JSON file with raw play data
    Returns: List of dictionaries of all the games and downloads the JSON files at folder_path
    
    Check if file is already downloaded.
        If yes, return the file path
        If not, check if it exists on the nhl api.
            If it does, save it locally and return the path
            If it doesn't, return None
    """

    file_path = folder_path + str(game_id) + '.json'
    if not os.path.isfile(file_path):
        data = requests.get(f'https://statsapi.web.nhl.com/api/v1/game/{game_id}/feed/live/')
        if (data.status_code == 404):
            return None
        with open(file_path, 'w') as f:
            json.dump(data.json(), f)

    return file_path


def get_season_files(seasons = [2015, 2016, 2017, 2018, 2019], folder_path = data_folder_path, clear_existing_files = False):
#def get_season_files(seasons = [2020], folder_path = data_folder_path, clear_existing_files = False):

    """download all regular season + playoff files from specified seasons into directory.
     Combine into one file, which is returned and saved at top of directory

     Note: might have to do one year at a time due to memory limitations"""

    if clear_existing_files:
        filelist = [ f for f in os.listdir(folder_path)  if f.endswith('.json') ]
        for f in filelist:
            os.remove(os.path.join(folder_path, f))

    result = list()

    for season in seasons:

        # get regular season games
        file_exists = True
        game_id = season * 1000000 + 20000 + 1 #id of first game in reg season
        while(file_exists):
            file = get_file(game_id, folder_path)
            if file is not None:
                with open(file, 'r') as f:
                    result.append(json.load(f))
            else:
                file_exists = False
            game_id +=1


        # get playoff games

        if season == 2019: # in 2019-2020 there was a playoff round 0 due to covid
            first_round = 0
        else:
            first_round = 1

        for playoff_round in range(first_round, 5):
            for matchup in range(1, 2**(4-playoff_round-1*(playoff_round==0))+1):
                file_exists = True
                game_id = season * 1000000 + 30000 + playoff_round * 100 + matchup * 10 + 1   # id of first game in playoff round & matchup
                while(file_exists):
                    file = get_file(game_id, folder_path)
                    if file is not None:
                        with open(file, 'r') as f:
                            result.append(json.load(f))
                    else:
                        file_exists = False
                    game_id +=1

    output_file_name = '0_all_games_' + "".join([str(season) for season in seasons]) + '.json'
    with open(folder_path + output_file_name, 'w') as f:
            json.dump(result, f)

    return result


def get_player_name(event, role):

    assert role in ['Goalie','Shooter','Scorer'], "selected role not supported"
    for player in event['players']:
        if (player['playerType'] == role):
            return player['player']['fullName']

def return_list_value_if_exists(lst, pos):
    if len(lst) > pos :
        return lst[pos]
    else:
        return None


def get_distance_from_net_and_side(df, net_distance_from_center = 89):

    """
    Compute the distance from where a shot was taken to the goal line



    source: https://community.rapidminer.com/discussion/44904/using-the-nhl-api-to-analyze-pro-ice-hockey-data-part-1
    """

    df['coordinates_multiplier'] = 1 -\
                                   2 * (df['home_team_side_1st_period']=='right') * (((df['attacking_team'] == df['home_team']) & df['period'].isin([1,3,5,7,9])) | ((df['attacking_team'] == df['away_team']) & df['period'].isin([2,4,6,8]))) -\
                                   2 * (df['home_team_side_1st_period']=='left') * (((df['attacking_team'] == df['home_team']) & df['period'].isin([2,4,6,8])) | ((df['attacking_team'] == df['away_team']) & df['period'].isin([1,3,5,7,9])))
    df['distance_from_net'] = df['home_team_side_1st_period'].isnull() * \
                                  np.minimum(
                                             ((df['x_coordinates'] - net_distance_from_center)**2 + (df['y_coordinates'] - 0)**2)**0.5,
                                             ((df['x_coordinates'] + net_distance_from_center)**2 + (df['y_coordinates'] - 0)**2)**0.5
                                  ) + \
                              ~df['home_team_side_1st_period'].isnull() * \
                                  ((df['x_coordinates'] - net_distance_from_center * df['coordinates_multiplier'])**2 + (df['y_coordinates'] - 0)**2)**0.5
    df['attacking_team_side'] = np.where(df['coordinates_multiplier'] == 1, 'left', 'right')
    del df['coordinates_multiplier']

    return df


def return_tidy_df(games, clear_games = True):

    data = []

    for game in games:

        game_id = game['gameData']['game']['pk']
        season = game['gameData']['game'].get('season')
        season_type = game['gameData']['game']['type']
        home_team = game['gameData']['teams']['home']['name']
        away_team = game['gameData']['teams']['away']['name']
        d = return_list_value_if_exists(game['liveData']['linescore']['periods'], 0)
        if d is not None:
            home_team_side_1st_period =  d['home'].get('rinkSide')
        else:
            home_team_side_1st_period = None

        events = game['liveData']['plays']['allPlays']

        for event in events:

            event_type = event['result']['event']

            if event_type == 'Shot':
                event_id = event['about']['eventIdx']
                attacking_team = event['team']['name']
                attacking_player = get_player_name(event, 'Shooter')
                goalie = get_player_name(event, 'Goalie')
                period = event['about']['period']
                period_time = event['about']['periodTime']
                goal_ind = 0
                shot_ind = 1
                x_coordinates = event['coordinates'].get('x')
                y_coordinates = event['coordinates'].get('y')
                shot_type = event['result'].get('secondaryType')
                empty_net = None
                strength = None
                gwg = None

                data.append([game_id, season, season_type, event_id, home_team, away_team, home_team_side_1st_period,
                             attacking_team, attacking_player, goalie, period, period_time, goal_ind,
                             shot_ind, x_coordinates, y_coordinates, shot_type, empty_net, strength, gwg])

            elif event_type == 'Goal':
                event_id = event['about']['eventIdx']
                attacking_team = event['team']['name']
                attacking_player = get_player_name(event, 'Scorer')
                goalie = get_player_name(event, 'Goalie')
                period = event['about']['period']
                period_time = event['about']['periodTime']
                goal_ind = 1
                shot_ind = 0
                x_coordinates = event['coordinates'].get('x')
                y_coordinates = event['coordinates'].get('y')
                shot_type = event['result'].get('secondaryType')
                empty_net = event['result'].get('emptyNet')
                strength = event['result']['strength']['name']
                gwg = event['result']['gameWinningGoal']

                data.append([game_id, season, season_type, event_id, home_team, away_team, home_team_side_1st_period,
                             attacking_team, attacking_player, goalie, period, period_time, goal_ind,
                             shot_ind, x_coordinates, y_coordinates, shot_type, empty_net, strength, gwg])

    df = pd.DataFrame(data,
                      columns=['game_id', 'season', 'season_type', 'event_id', 'home_team', 'away_team', 'home_team_side_1st_period',
                               'attacking_team', 'attacking_player', 'goalie', 'period', 'period_time','goal_ind',
                               'shot_ind', 'x_coordinates', 'y_coordinates', 'shot_type', 'empty_net', 'strength', 'gwg'])

    df = get_distance_from_net_and_side(df, net_distance_from_center = 89)

    if clear_games:
        del games

    return df


def tidy_df_loop(seasons = [2015, 2016, 2017, 2018, 2019]):
#def tidy_df_loop(seasons = [2020]):

    """loop over all season. for each season, get file, create tidy df from it, and clear the memory.
       append the dfs together"""
    i = 0
    for season in seasons:
        data_files = get_season_files(seasons = [season])

        if i == 0:
            df = return_tidy_df(data_files)
        else:
            sub_df = return_tidy_df(data_files)
            df = pd.concat([df, sub_df])
            del sub_df
        i += 1
        print(f"finished {season} season")

    return df


# 4. Tidy Data

Refer to ift6758/data/question_2_4.py for functions used to import the data.

This takes a while to run, so run it only once, save output as csv and import the csv for future uses

In [11]:
df = tidy_df_loop()

finished 2015 season
finished 2016 season
finished 2017 season
finished 2018 season
finished 2019 season


In [12]:
df.to_csv('tidy_df.csv', index=False)

## 4.1

In your blog post, include a small snippet of your final dataframe (e.g. using head(10)). You can just include a screenshot rather than fighting to get the tables neatly formatted in HTML/markdown.

In [6]:
import json
import pandas as pd
df = pd.read_csv('tidy_df.csv')

In [7]:
df.sample(10)

,game_id,season,season_type,event_id,home_team,away_team,home_team_side_1st_period,attacking_team,attacking_player,goalie,...,goal_ind,shot_ind,x_coordinates,y_coordinates,shot_type,empty_net,strength,gwg,distance_from_net,attacking_team_side
28382,2020020469,20202021,R,217,New York Islanders,Philadelphia Flyers,left,New York Islanders,Michael Dal Colle,Carter Hart,...,1,0,70.0,9.0,Wrist Shot,False,Even,False,21.023796,left
56141,2020030231,20202021,P,168,Colorado Avalanche,Vegas Golden Knights,right,Colorado Avalanche,Nathan MacKinnon,Robin Lehner,...,0,1,64.0,27.0,Slap Shot,NaN,NaN,NaN,36.796739,left
25327,2020020418,20202021,R,257,Philadelphia Flyers,Washington Capitals,right,Philadelphia Flyers,Scott Laughton,Ilya Samsonov,...,1,0,-61.0,-14.0,Wrist Shot,False,Even,False,31.304952,right
31916,2020020528,20202021,R,133,Arizona Coyotes,San Jose Sharks,left,Arizona Coyotes,Dryden Hunt,Martin Jones,...,0,1,-66.0,7.0,Wrist Shot,NaN,NaN,NaN,24.041631,right
56307,2020030234,20202021,P,42,Vegas Golden Knights,Colorado Avalanche,NaN,Colorado Avalanche,Brandon Saad,Marc-Andre Fleury,...,0,1,67.0,-29.0,Wrist Shot,NaN,NaN,NaN,36.400549,left
57105,2020030321,20202021,P,253,Tampa Bay Lightning,New York Islanders,left,Tampa Bay Lightning,Anthony Cirelli,Semyon Varlamov,...,0,1,75.0,0.0,Tip-In,NaN,NaN,NaN,14.000000,left
52151,2020020864,20202021,R,179,Vancouver Canucks,Calgary Flames,right,Vancouver Canucks,Bo Horvat,Louis Domingue,...,0,1,54.0,-19.0,Wrist Shot,NaN,NaN,NaN,39.824616,left
50936,2020020843,20202021,R,291,Chicago Blackhawks,Dallas Stars,right,Dallas Stars,Jamie Benn,Kevin Lankinen,...,0,1,84.0,-19.0,Wrist Shot,NaN,NaN,NaN,19.646883,left
41167,2020020681,20202021,R,195,Detroit Red Wings,Chicago Blackhawks,right,Detroit Red Wings,Jakub Vrana,Kevin Lankinen,...,0,1,-66.0,-11.0,Snap Shot,NaN,NaN,NaN,25.495098,right
22700,2020020374,20202021,R,152,New Jersey Devils,New York Rangers,left,New York Rangers,Pavel Buchnevich,Mackenzie Blackwood,...,0,1,75.0,-20.0,Snap Shot,NaN,NaN,NaN,24.413111,left
